In [8]:
import os
from glob import glob
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2

In [40]:
vlat_data = {
    "TreeMap": {
        "question": "eBay is nested in the Software category.",
        "answer": "False",
        "image_path": "TreeMap.png",
        "options": ["True", "False"]
    },
    "Stacked100": {
        "question": "Which country has the lowest proportion of Gold medals?",
        "answer": "Great Britain",
        "image_path": "Stacked100.png",
        "options": ["Great Britain", "USA", "Japan", "Australia"]
    },
    "Histogram": {
        "question": "What distance have customers traveled in the taxi the most?",
        "answer": "30-40km",
        "image_path": "Histogram.png",
        "options": ["50-70km", "30-40km", "20-30km", "50-60km"]
    },
    "Choropleth": {
        "question": "In 2020, the unemployment rate for Washington (WA) was higher than that of Wisconsin (WI)?",
        "answer": "True",
        "image_path": "Choropleth.png",
        "options": ["True", "False"]
    },
    "PieChart": {
        "question": "What is the approximate global smartphone market share of Samsung?",
        "answer": "17.6%",
        "image_path": "PieChart.png",
        "options": ["17.6%", "25.3%", "10.9%", "35.2%"]
    },
    "BubbleChart": {
        "question": "Which city's metro system has the largest number of stations?",
        "answer": "Shanghai",
        "image_path": "BubbleChart.png",
        "options": ["Beijing", "Shanghai", "London", "Seoul"]
    },
    "StackedBar": {
        "question": "What is the cost of peanuts in Seoul?",
        "answer": "$6.1",
        "image_path": "StackedBar.png",
        "options": ["$5.2", "$6.1", "$7.5", "$4.5"]
    },
    "LineChart": {
        "question": "What was the price of a barrel of oil in February 2020?",
        "answer": "$50.54",
        "image_path": "LineChart.png",
        "options": ["$50.54", "$47.02", "$42.34", "$43.48"]
    },
    "BarChart": {
        "question": "What is the average internet speed in Japan?",
        "answer": "40.51 Mbps",
        "image_path": "BarChart.png",
        "options": ["42.30 Mbps", "40.51 Mbps", "35.25 Mbps", "16.16 Mbps"]
    },
    "AreaChart": {
        "question": "What was the average price of a pound of coffee in October 2019?",
        "answer": "$0.71",
        "image_path": "AreaChart.png",
        "options": ["$0.71", "$0.90", "$0.80", "$0.63"]
    },
    "StackedArea": {
        "question": "What was the ratio of girls named \"Isla\" to girls named \"Amelia\" in 2012 in the UK?",
        "answer": "1 to 2",
        "image_path": "StackedArea.png",
        "options": ["1 to 1", "1 to 2", "1 to 3", "1 to 4"]
    },
    "Scatterplot": {
        "question": "There is a negative relationship between the height and the weight of the 85 males.",
        "answer": "False",
        "image_path": "Scatterplot.png",
        "options": ["True", "False"]
    }
}

additional_data = {
    "TreeMap": {
        "question": "For which website was the number of visitors the largest?",
        "answer": "Google",
        "image_path": "TreeMap.png",
        "options": ["Facebook", "Ask", "Google", "eBay"]
    },
    "Stacked100": {
        "question": "Japan had higher proportion of Silver medals than Australia.",
        "answer": "True",
        "image_path": "Stacked100.png",
        "options": ["True", "False"]
    },
    "Histogram": {
        "question": "More people have rated the taxi between 4.6 and 4.8 than between 4.2 and 4.4",
        "answer": "True",
        "image_path": "Histogram.png",
        "options": ["True", "False"]
    },
    "Choropleth": {
        "question": "In which state was the unemployment rate the highest in 2015?",
        "answer": "New Mexico (NM)",
        "image_path": "Choropleth.png",
        "options": ["Alaska (AK)", "New Mexico (NM)", "Florida (FL)", "New York (NY)"]
    },
    "PieChart": {
        "question": "The global smartphone market share of Apple is larger than that of Huawei.",
        "answer": "True",
        "image_path": "PieChart.png",
        "options": ["True", "False"]
    },
    "BubbleChart": {
        "question": "What is the range of the total length of the metro systems?",
        "answer": "150 - 600 km",
        "image_path": "BubbleChart.png",
        "options": ["150 - 600 km", "240 - 380 km", "240 - 560 km", "180 - 560 km"]
    },
    "StackedBar": {
        "question": "The cost of vodka in Atlanta is higher than that of Honolulu.",
        "answer": "True",
        "image_path": "StackedBar.png",
        "options": ["True", "False"]
    },
    "LineChart": {
        "question": "Over the course of the second half of 2015, the price of a barrel of oil was ____________.",
        "answer": "falling",
        "image_path": "LineChart.png",
        "options": ["rising", "falling", "staying the same"]
    },
    "BarChart": {
        "question": "How many countries in Asia is the average internet speed slower than Thailand?",
        "answer": "6",
        "image_path": "BarChart.png",
        "options": ["5", "6", "7", "8"]
    },
    "AreaChart": {
        "question": "What was the range of the average price of a pound of coffee beans between January 2013 and December 2014?",
        "answer": "$4.6 - $6.0",
        "image_path": "AreaChart.png",
        "options": ["$4.4 - $6.2", "$4.6 - $5.9", "$4.6 - $6.0", "$4.6 - $6.1"]
    },
    "StackedArea": {
        "question": "In the UK, the number of girls named \"Amelia\" in 2014 was more than it was in 2013,",
        "answer": "False",
        "image_path": "StackedArea.png",
        "options": ["True", "False"]
    },
    "Scatterplot": {
        "question": "What is the weight for the person who is 165.1cm tall?",
        "answer": "70.5 kg",
        "image_path": "Scatterplot.png",
        "options": ["53.9 kg", "67.7 kg", "70.5 kg", "82.7 kg"]
    }
}

In [41]:
def static_grids(img, grid_count=8):
    image_height, image_width = img.shape[:2]
    interval_y = int(np.ceil(image_height / (grid_count+1)))
    interval_x = int(np.ceil(image_width / (grid_count+1)))
    x_grids = []
    y_grids = []
    
    for i in range(interval_x, image_width, interval_x):
        x_grids.append(i)
    for i in range(interval_y, image_height, interval_y):
        y_grids.append(i)
    
    return x_grids, y_grids

In [85]:
def create_base_components():
    return {
        "grid": {
            "type": "react-component",
            "path": "literacy/assets/grid.jsx",
            "instructionLocation": "sidebar",
            "nextButtonLocation": "sidebar",
            "nextButtonDisableTime": 90000,
            "nextButtonEnableTime": 5000
        },
    }

In [ ]:

def create_default_components():
    return {
        # "welcome": {
        #     "type": "markdown",
        #     "path": "literacy/assets/welcome.md",
        #     "response": []
        # },
        "instructions": {
            "type": "markdown",
            "path": "literacy/assets/instructions.md",
            "response": []
        },  
        "consent": {
            "type": "markdown",
            "path": "literacy/assets/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "vlat_intro": {
            "type": "markdown",
            "path": "literacy/assets/vlat_intro.md",
            "response": []
        },
        "demographics": {
            "type": "markdown",
            "path": "literacy/assets/blank.md",
            "response": [
                {
                    "id": "gender",
                    "prompt": "What is your **gender**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Woman",
                        "Man",
                        "Prefer not to say"
                    ],
                    "withDivider": True

                },
                {
                    "id": "age",
                    "prompt": "What is your **age**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "Under 18 years",
                        "18-24 years",
                        "25-34 years",
                        "35-44 years",
                        "45-54 years",
                        "55-64 years",
                        "65 years or older",
                        "Prefer not to say"
                    ],
                    "withDivider": True
                },
                {
                    "id": "education",
                    "prompt": "What is the **highest degree or level of education** you have completed?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Less than high school",
                        "High school diploma or equivalent",
                        "Bachelor's degree or equivalent",
                        "Master's degree or equivalent",
                        "Doctoral degree or equivalent"
                    ],
                    "withDivider": True
                }
            ]
        },
        "NASA": {
            "type": "markdown",
            "path": "literacy/assets/blank.md",
            "response": [
                {
                    "id": "mental-demand",
                    "prompt": "How **mentally demanding** was the task?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Very High",
                    "leftLabel": "Very Low",
                    "withDivider": True,
                },
                {
                    "id": "physical-demand",
                    "prompt": "How **physically demanding** was the task?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Very High",
                    "leftLabel": "Very Low",
                    "withDivider": True
                },
                {
                    "id": "temporal-demand",
                    "prompt": "How **hurried or rushed** was the pace of the task?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Very High",
                    "leftLabel": "Very Low",
                    "withDivider": True
                },
                {
                    "id": "performance",
                    "prompt": "How **successful** were you in accomplishing what you were asked to do?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Perfect",
                    "leftLabel": "Failure",
                    "withDivider": True
                },
                {
                    "id": "effort",
                    "prompt": "How **hard** did you have to work to accomplish your level of performance?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Very High",
                    "leftLabel": "Very Low",
                    "withDivider": True
                },
                {
                    "id": "frustration",
                    "prompt": "How **insecure, discouraged, irritated, stressed, and annoyed** were you?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 7,
                    "rightLabel": "Very High",
                    "leftLabel": "Very Low",
                    "withDivider": True
                }
            ]
        }
    }


def create_bubble_components(data):
    tasks = dict()
    grid_count = 9
    for chart_type, value in data.items():
        question = value["question"]
        image_path = value["image_path"]
        options = value["options"]
        answer = value["answer"]
        img = cv2.imread(f"assets/images/{image_path}")
        x_grids, y_grids = static_grids(img, grid_count=grid_count)

        tasks[f"bubble-{image_path}"] = {
            "baseComponent": "grid",
            "instruction": f" **Q: {question}**",
            "parameters": {
                "image": f"../literacy/assets/images/{image_path}",
                "question": question,
                "x_grids": x_grids,
                "y_grids": y_grids,
            },
            "response": [
                {
                    "id": f"{chart_type}-original",
                    "prompt": "Your answer",
                    "required": True,
                    "location": "sidebar",
                    "type": "radio",
                    "options": options
                },
            ],
            "correctAnswer": [
                {
                "id": f"{chart_type}-original",
                "answer": answer
                }
            ]
        }

    return tasks

In [87]:
def sequence_generator(basic_components, additional_components):
    sequence = {
        "order": "fixed",
        "components": [
            "consent",
            "vlat_intro",
            "$mini-vlat.se.full",
            "instructions",
            {
                "id": "same_inst",
                "order": "random",
                "numSamples": 6,
                "components": list(basic_components.keys())
            },
            {
                "id": "different_inst",
                "order": "random",
                "numSamples": 6,
                "components": list(additional_components.keys())
            },
            "NASA",
            "demographics"
        ]
    }
    return sequence

In [88]:
prolificRedirection = "https://app.prolific.com/submissions/complete?cc=C1DEBJ8K"

In [89]:
basic_components = create_bubble_components(vlat_data)
additional_components = create_bubble_components(additional_data)

In [90]:
default_components = create_default_components()
baseComponents = create_base_components()
components = default_components | basic_components | additional_components
sequence = sequence_generator(basic_components, additional_components)
print(f"Total number of components: {len(components)}")

Total number of components: 17


In [91]:
with open("config.json", "r") as f:
    config = json.load(f)
config['uiConfig']['studyEndMsg'] = f"**Thank you for completing the study. You may click this link and return to Prolific**: [{prolificRedirection}]({prolificRedirection})"
config['components'] = components
config['sequence'] = sequence
config['baseComponents'] = baseComponents
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)